In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import os
import calendar
import math

In [53]:
#paths to kiki and will data 
k_disc_path = r"C:\Users\willd\OneDrive\Desktop\Budget\statements\kiki_disc"
k_chase_path = r"C:\Users\willd\OneDrive\Desktop\Budget\statements\kiki_chase"
k_ally_path = r"C:\Users\willd\OneDrive\Desktop\Budget\statements\kiki_ally"
# k_end_path = r"C:\Users\willd\OneDrive\Desktop\Budget\clean_data\k_data\k_budget_data.xlsx"
w_pnc_path = r"C:\Users\willd\OneDrive\Desktop\Budget\statements\pnc_csvs"
w_chase_path = r"C:\Users\willd\OneDrive\Desktop\Budget\statements\chase_csvs"
end_path = r"C:\Users\willd\OneDrive\Desktop\Budget\clean_data\combined_budget_data.xlsx"

## Functions

In [62]:
# combine all csvs in a path return df  // with no duplicates 

def combine_csvs_pnc(path):
    df_list = []
    for item in os.listdir(path):
        full_path = os.path.join(path, item)
        df = pd.read_csv(full_path)
        df_list.append(df)
    combined = pd.concat(df_list)
    combined.columns.values[0] = "Date"
    no_dup = combined.drop_duplicates(subset=['Date', 'Description'], keep='first', inplace=False, ignore_index=True)
    return no_dup



def combine_csvs(path):
    df_list = []
    for item in os.listdir(path):
        full_path = os.path.join(path, item)
        df = pd.read_csv(full_path)
        df_list.append(df)
    combined = pd.concat(df_list, ignore_index=True)
#     combined = combined.drop_duplicates(keep='first')
    combined.columns.values[0] = "Date"
    no_dup = combined.drop_duplicates(keep='first', inplace=False, ignore_index=True)
    return no_dup


# remove specific payments from a df
def remove_item(df, descript):
    remove_df = df[~df['Description'].str.contains(descript)]
    return remove_df
    
    
# reformat to only date amount desc, category columns
def column_remove(df, user, bank):
    df.columns = df.columns.str.replace(' ', '') 
    if 'Category' not in df.columns:
        df["Category"] = 'misc'
    df["User"] = user
    df["Bank"] = bank
#     df["In/Out"] = 'tbd'
    new = df[['Date', 'Amount', 'Description', 'Category',"User","Bank"]]
    return new

# assign transaction catagory
def k_category_assign(df):
    # make seriess 
    s1 = df['Description']
    s2 = df['Amount']
    #ks category conditions and values
    k_cat_condition = [(s1.str.contains('VENMO'))&(s2 <= (-680)) & (s2 >= (-850)),(s1.str.contains('VENMO')) & (s2 > (0)),
                       (s1.str.contains('VENMO')),(s1.str.contains('COMCAST')),
                       (s1.str.contains('Primco')), (s1.str.contains('SCHWAB')), (s1.str.contains('Interest Paid'))]
    k_cat_value = ['RENT','ven in','K_VENMO','Bills & Utilities', 'Prim and Co', 'SCHWAB', 'Intrest']
    df['Category'] = np.select(k_cat_condition, k_cat_value)
    df.loc[df['Category'] == '0', 'Category'] = 'Misc'
    return df 

# math.isclose(k_row['in'], row['out'], rel_tol=threshhold 


# fill this into k rent situation




def w_category_assign(w_pnc):
    # make seriess 
    df = w_pnc['Description']
    df2 = w_pnc['Amount']
    cat_condition = [(df.str.contains('VENMO')) & (df2 < 600),(df.str.contains('VENMO'))& (df2 > 0) & (df2 > 600), (df.str.contains('SPOTIFY') | df.str.contains('APPLE')),
                     (df.str.contains('ACTALENT')), (df.str.contains('CHECK'))
                    , (df.str.contains('SCHWAB BROK'))]
    cat_value = ['W_VENMO','K_RENT', 'SUBSCRIP', 'Belair', 'W_RENT', 'SCHWAB']
    w_pnc['Category'] = np.select(cat_condition, cat_value)
    return w_pnc

# , (df.str.contains('VENMO')) & (df2 < 0), 'VENMO OUT',  & (df2 > 0)

# function to fill ammount column in the pnc dataframe 
def amount_fill(data):
    if '$' in str(data['Withdrawals']):
        amount = '-' + data['Withdrawals']
    else:
        amount = data['Deposits']
    return float(amount.replace('$', '').replace(',', ''))

# match venmo transaction by a threshhold 
def venmatch(combo, threshhold): 
    tup_list = []
    ven = combo[combo['Category']== 'VENMO']
    w_ven = ven[ven['User'] == 'W']
    k_ven = ven[ven['User'] == 'K']
    for w_index , row in w_ven.iterrows():
        for k_index , k_row in k_ven.iterrows():
            if math.isclose(k_row['in'], row['out'], rel_tol=threshhold):
                new_val = k_row['in'] - row['out']
                tup = (w_index, k_index, new_val)
                tup_list.append(tup)
    return tup_list


# adjust venmo payments
def ven_mod(df, tup_list):
    for item in tup_list:
        df.iloc[item[1], df.columns.get_loc('in')] = item[2]
        df = df.drop(df.index[item[0]])
    return df

def remove_row(df, amount, date):
    r_index = df.index[(df['Amount'] == amount)&(df['Date'] == date)]
    df = df.drop(index=r_index[0])
    return df

# change categorie by description
def category_change(df, desc, cat):
    df.loc[df['Description'].str.contains(desc), "Category"] = cat
    return df

# def category_change_by_date(df, desc, cat):
#     df.loc[df['Description'].str.contains(desc), "Category"] = cat
#     return df

def category_by_amount_date(df, amount, date, cat):
    r_index = df.index[(df['Amount'] == amount)&(df['Date'] == date)]
    df.loc[r_index[0], "Category"] = cat
    return df

#     print(r_index[0])

# # adjust venmo payments
# def ven_mod(df, tup_list):
#     for item in tup_list:
#         df.iloc[item[1], df.columns.get_loc('in')] = item[2]
#         df = df.drop(df.index[item[0]])
#     return item[0]

In [63]:
#compile all files in each folder
w_pnc = combine_csvs_pnc(w_pnc_path)
#make one row of just amount from withdrawls and eposits
w_pnc['Amount'] = w_pnc.apply(amount_fill, axis=1)
w_pnc = column_remove(w_pnc, 'W', 'PNC')
w_pnc = w_pnc.drop_duplicates()
w_chase = column_remove(combine_csvs(w_chase_path), 'W', 'chase')
# add catacories to pnc data frame
w_pnc = w_category_assign(w_pnc)
# make a misc catagory of remaining  
w_pnc.loc[w_pnc['Category'] == '0', 'Category'] = 'Misc'

# w_pnc = w_pnc[w_pnc['Category'] != 'VENMO']
# remove CC Payments from both 
w_pnc = w_pnc[~w_pnc['Description'].str.contains('CHASE CREDIT CRD EPA')]
w_chase = w_chase[w_chase['Amount'] < 0]
# remove pnc transfers 
w_pnc = w_pnc[~w_pnc['Description'].str.contains('|'.join(['ONLINE TRANSFER', 'ALLY BANK']))]

k_disc = column_remove(combine_csvs(k_disc_path), 'K', 'disc')
k_chase = column_remove(combine_csvs(k_chase_path), 'K', 'chase')
k_ally = column_remove(combine_csvs(k_ally_path), 'K', 'ally')
# categorize ally transactions
k_ally = k_category_assign(k_ally)
k_ally = k_ally[k_ally['Category'] != 'ven in']
#remove cc payments from ally
k_ally = remove_item(k_ally, 'CHASE CREDIT CRD')
k_chase = remove_item(k_chase, 'AUTOMATIC PAYMENT')
k_disc = remove_item(k_disc, 'DIRECTPAY FULL BALANCESEE DETAILS')
# make remoining discover 
k_disc['Amount'] = 0 - k_disc['Amount']

In [70]:
# combine pnc and chase for analysis 
# df_list = [w_pnc, w_chase, k_disc, k_chase,k_ally]
full_df = pd.concat([w_chase, k_disc, k_chase,k_ally, w_pnc])

# change date dtype
full_df['Date'] = pd.to_datetime(full_df['Date'])
# add month column 
full_df['Year'] = pd.DatetimeIndex(full_df['Date']).year
# add month column 
full_df['Month'] = pd.DatetimeIndex(full_df['Date']).month
full_df['m_y'] = full_df['Date'].dt.to_period('M')
# change month colum to date
full_df['Month'] = full_df['Month'].apply(lambda x: calendar.month_abbr[x])
full_df = full_df.sort_values(by='Date')

month_list = full_df['Month'].unique()
for mon1 in month_list:
    month_df = full_df[full_df['Month'] == mon1]
    k_rent = month_df.loc[month_df['Category'] == 'K_RENT', 'Amount']
    w_rent = month_df.loc[month_df['Category'] == 'W_RENT', 'Amount']  
    if (len(k_rent) > 0) & (len(w_rent) > 0):
        w_index = w_rent.index
        w_index = w_index[0]
        rent_actual = w_rent.iloc[0] + k_rent.iloc[0]
#         print(rent_actual, mon1)
        full_df.loc[(full_df['Category'] == 'W_RENT')&(full_df['Month'] == mon1), 'Amount'] = rent_actual
full_df = full_df[full_df['Category'] != 'K_RENT']

full_df.loc[full_df['Category'] == "W_RENT", "Category"] = 'RENT'


# remove specific rows by amount and date 

full_df = remove_row(full_df, 2000, '02/26/2024')
full_df = remove_row(full_df, 231.57, '03/28/2024')
full_df = category_change(full_df,'PROGRESSIVE','CAR INSURANCE')
# change investment 
full_df = category_by_amount_date(full_df, -1000, '03/04/2024', 'Other Investments')
#change to rent
full_df = category_by_amount_date(full_df, -231.57, '03/28/2024', 'RENT')


# dfs of income and expenses 
money_in = full_df[full_df['Amount'] > 0]
money_out = full_df[full_df['Amount'] < 0]
money_out.loc[money_out['Amount'] < 0, 'Amount'] = 0 - money_out['Amount']

#income expense combo df 
combo = full_df
combo[['in','out']] = 0
combo['in'][(combo['Amount'] > 0)] = combo['Amount']
combo['out'][(combo['Amount'] < 0)] = (0 - combo['Amount'])
combo = combo.reset_index(drop=True)

#remove venmo
# list1 = venmatch(combo, 0.01)
# new_df = ven_mod(combo, list1)


# df of expense to income per month 
month_list = full_df['m_y'].unique()
dict_list = []
for mon in month_list:
    df_in = money_in[money_in['m_y'] == mon]
    df_out = money_out[money_out['m_y'] == mon]
    month_in = df_in['Amount'].sum()
    month_out = df_out['Amount'].sum()
    dict1 = {'Month': mon, 'Money In': month_in, 'Money Out': month_out}
    dict_list.append(dict1)
monthly_ratio = pd.DataFrame(dict_list)



C:\Users\willd\AppData\Local\Temp\ipykernel_12888\1980534954.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  money_out.loc[money_out['Amount'] < 0, 'Amount'] = 0 - money_out['Amount']
C:\Users\willd\AppData\Local\Temp\ipykernel_12888\1980534954.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combo['in'][(combo['Amount'] > 0)] = combo['Amount']
C:\Users\willd\AppData\Local\Temp\ipykernel_12888\1980534954.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.

### Export to csv

In [74]:
writer = pd.ExcelWriter(end_path, engine="xlsxwriter")
combo.to_excel(writer, sheet_name="full_df")
monthly_ratio.to_excel(writer, sheet_name="all_mon")
money_in.to_excel(writer, sheet_name="income")
money_out.to_excel(writer, sheet_name="expense")
writer.close()

In [28]:
# full_df[full_df['Date'] == '02/26/2024']

In [73]:
# combo[combo['Date'] == '03/28/2024']

In [32]:


# adjust venmo payments
def ven_mod(df, tup_list):
    for item in tup_list:
        df.iloc[item[1], df.columns.get_loc('in')] = item[2]
        new = df.drop(df.index[item[0]])
    return new

In [27]:
# list2 = venmatch(combo, 0.1)
# new_df = ven_mod(combo, list2)
# new_df[new_df['Category']== 'VENMO']

In [11]:
test4 = venmatch(df, 0.1)
test4

[(794, 793, 84.66999999999996),
 (794, 860, 39.16999999999996),
 (859, 793, 45.5),
 (859, 860, 0.0),
 (953, 962, 35.0),
 (1088, 298, 7.430000000000007)]

In [23]:
# ven = new_df[new_df['Category']== 'VENMO']
# ven

In [24]:
# ven = test3[test3['Category']== 'VENMO']
# ven

In [28]:
# writer = pd.ExcelWriter(end_path, engine="xlsxwriter")
# full_df.to_excel(writer, sheet_name="full_df")
# # monthly_ratio.to_excel(writer, sheet_name="all_mon")
# # money_in.to_excel(writer, sheet_name="income")
# # money_out.to_excel(writer, sheet_name="expense")
# writer.close()